In [2]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(136)
cluster

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [3]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://172.30.100.1:44895 Dashboard: http://172.30.100.1:45212/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8204400     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204401     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204402     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204403     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204404     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204405     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204406     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204407     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204408     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204409     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204410     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8204411     hsw24 make_pro albert7a

In [ ]:
import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

In [ ]:
!squeue -u albert7a

In [ ]:
import os
import sys
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import cmocean
import glob

%matplotlib inline


In [ ]:
#params

time_serie_eke=True
time_serie_surf_tke=True
time_serie_0_100m_tke=True


In [ ]:
config='EU36'
case='MPC001'
freq='1d'
#year='2010'

datadir='/scratch/cnt0024/hmg2840/colombo/'+str(config)+'/'+str(config)+'-'+str(case)+'-S/'+str(freq)+'/'
#!ls $datadir
gridfile='/scratch/cnt0024/hmg2840/colombo/EU36/EU36-I/coordinates_EU36.nc'
maskfile='/scratch/cnt0024/hmg2840/colombo/EU36/EU36-I/EU36.L75_mesh_mask.nc'

In [ ]:
dsgrid=xr.open_dataset(gridfile)
dsmask=xr.open_dataset(maskfile)

In [ ]:
def compute_EKE(u,v,u2mean,v2mean):
    ua=0.5*((u2mean-u*u)+(u2mean.shift(x=-1)-u.shift(x=-1)*u.shift(x=-1)))
    va=0.5*((v2mean-v*v)+(v2mean.shift(y=-1)-v.shift(y=-1)*v.shift(y=-1)))
    uat=ua.rename({'depthu':'deptht'})
    vat=va.rename({'depthv':'deptht'})
    eke=0.5*(uat+vat)
    return eke


In [ ]:
def compute_TKE(u,v):
    ua=0.5*(u*u+u.shift(x=-1)*u.shift(x=-1))
    va=0.5*(v*v+v.shift(y=-1)*v.shift(y=-1))
    uat=ua.rename({'depthu':'deptht'})
    vat=va.rename({'depthv':'deptht'})
    eke=0.5*(uat+vat)
    return eke


In [ ]:
if not os.path.exists('Plots/'+config+'/Time_Series/'+config+'-'+case+'_surf-mean-tke.png'):
    filesU=sorted(glob.glob(datadir+'*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridU.nc'))
    filesV=sorted(glob.glob(datadir+'*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridV.nc'))
    dsU=xr.open_mfdataset(filesU, chunks={'x':700,'y':1000,'time_counter':1,'depthu':1})
    dsV=xr.open_mfdataset(filesV, chunks={'x':700,'y':1000,'time_counter':1,'depthv':1})
    u=dsU.vozocrtx
    v=dsV.vomecrty
    mask=dsmask.tmask[0]
    tke=compute_TKE(u,v)
    ts0=10000*tke[:,0].mean(dim='x').mean(dim='y')
    fig, ax = plt.subplots(1,1,figsize=(15,10))
    ts0.plot()
    plt.savefig('Plots/'+config+'/Time_Series/'+config+'-'+case+'_surf-mean-tke.png')

In [ ]:
if not os.path.exists('Plots/'+config+'/Time_Series/'+config+'-'+case+'_0-100m-mean-tke.png'):
    filesU=sorted(glob.glob(datadir+'*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridU.nc'))
    filesV=sorted(glob.glob(datadir+'*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridV.nc'))
    dsU=xr.open_mfdataset(filesU, chunks={'x':700,'y':1000,'time_counter':1,'depthu':1})
    dsV=xr.open_mfdataset(filesV, chunks={'x':700,'y':1000,'time_counter':1,'depthv':1})
    u=dsU.vozocrtx
    v=dsV.vomecrty
    mask=dsmask.tmask[0]
    tke=compute_TKE(u,v)
    ts0=10000*tke.sel(deptht=slice(0,100)).mean(dim='x').mean(dim='y')
    fig, ax = plt.subplots(1,1,figsize=(15,10))
    ts0.plot()
    plt.savefig('Plots/'+config+'/Time_Series/'+config+'-'+case+'_0-100m-mean-tke.png)